In [2]:
import os
import torch
from detectron2.data.datasets import register_coco_instances
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.catalog import MetadataCatalog
import cv2
import numpy as np
from pathlib import Path
import random
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf

from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import time
import datetime
import logging
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

from detectron2.config import get_cfg
cfg = get_cfg()

2024-01-11 19:30:06.975182: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 19:30:07.265344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-11 19:30:07.265426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-11 19:30:07.317570: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-11 19:30:07.412455: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 19:30:07.413366: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#Definition of functions for hail model training and inference

def register_coco():
    #Register train data set
    register_coco_instances("train_hail", {}, "./data/hail_20210620_r1/train/annotations/instances_default.json", "./data/hail_20210620_r1/train/images")
    dataset_dicts_train = DatasetCatalog.get("train_hail")
    hail_metadata_train = MetadataCatalog.get("train_hail")

    #Register validation data set
    register_coco_instances("val_hail", {}, "./data/hail_20210620_r1/val/annotations/instances_default.json", "./data/hail_20210620_r1/val/images")
    dataset_dicts_val = DatasetCatalog.get("val_hail")
    hail_metadata_val = MetadataCatalog.get("val_hail")

    #Register test data set
    register_coco_instances("test_hail", {}, "./data/hail_20210620_r1/test/lai/annotations/instances_default.json", "./data/hail_20210620_r1/test/lai/images")
    dataset_dicts_test = DatasetCatalog.get("test_hail")
    hail_metadata_test = MetadataCatalog.get("test_hail")

    
def main():
    HP_BASE_LR = hp.HParam('base_lr', hp.Discrete([0.0001, 0.00025, 0.0005, 0.001]))
    HP_GAMMA = hp.HParam('gamma', hp.Discrete([0.1,0.5]))
    HP_BATCH_SIZE_PER_IMAGE = hp.HParam('batch_size_per_image', hp.Discrete([128, 256]))
    
    AP_BBOX = 'AP_bbox'
    AP50_BBOX = 'AP50_bbox'
    AP75_BBOX = 'AP75_bbox'

    AP_SEGM = 'AP_segm'
    AP50_SEGM = 'AP50_segm'
    AP75_SEGM = 'AP75_segm'

    with tf.summary.create_file_writer('output/logs/hparam_tuning').as_default():
        hp.hparams_config(
            hparams=[HP_BASE_LR, HP_GAMMA, HP_BATCH_SIZE_PER_IMAGE],
            metrics=[hp.Metric(AP_BBOX, display_name='AP_bbox'), 
                     hp.Metric(AP50_BBOX, display_name='AP50_bbox'), 
                     hp.Metric(AP75_BBOX, display_name='AP75_bbox'),
                 hp.Metric(AP_SEGM, display_name='AP_segm'), 
                 hp.Metric(AP50_SEGM, display_name='AP50_segm'), 
                 hp.Metric(AP75_SEGM, display_name='AP75_segm')
                 ],
      )

    session_num = 0
    for base_lr in HP_BASE_LR.domain.values:
        for gamma in HP_GAMMA.domain.values:
            for batch_size_per_image in HP_BATCH_SIZE_PER_IMAGE.domain.values:
                hparams = {
                    HP_BASE_LR: base_lr,
                    HP_GAMMA: gamma,
                    HP_BATCH_SIZE_PER_IMAGE: batch_size_per_image,
                    }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                #run('output/logs/hparam_tuning/' + run_name, hparams)
                session_num += 1
                
    return hparams        

def train_hail_model(hparams, run_dir):
    cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    
    cfg.INPUT.MIN_SIZE_TRAIN = (500,)
    cfg.INPUT.MIN_SIZE_TRAIN = 0
    
    cfg.OUTPUT_DIR = run_dir
    
    cfg.DATASETS.TRAIN = ("train_hail",)
    cfg.DATASETS.TEST = ("val_hail",)
    
    cfg.DATALOADER.NUM_WORKERS = 1

    #cfg.INPUT.RANDOM_FLIP = "horizontal"
    #cfg.SOLVER.CHECKPOINT_PERIOD = 2000

    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = hparams[HP_BASE_LR]
    cfg.SOLVER.WARMUP_ITERS = 300
    cfg.SOLVER.MAX_ITER = 3000 #adjust up if val AP is still rising, adjust down if overfit
    cfg.SOLVER.STEPS = (2400, 2700)
    cfg.SOLVER.GAMMA = hparams[HP_GAMMA]

    # Test
    cfg.TEST.EVAL_PERIOD = 100
    cfg.TEST.DETECTIONS_PER_IMAGE = 50

    cfg.MODEL.WEIGHTS = "model_final_f10217.pkl"  # initialize from model zoo
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = hparams[HP_BATCH_SIZE_PER_IMAGE] 
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 class (hail)
    cfg.MODEL.MASK_ON = True
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = CocoTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    output_dir = cfg.OUTPUT_DIR + '/eval/'
    os.makedirs(output_dir, exist_ok=True)
    evaluator = COCOEvaluator("val_hail", cfg, False, output_dir=output_dir)
    val_loader = build_detection_test_loader(cfg, "val_hail")
    result = inference_on_dataset(trainer.model, val_loader, evaluator)
    
    AP_bbox = result['bbox']['AP']
    AP50_bbox = result['bbox']['AP50']
    AP75_bbox = result['bbox']['AP75']
    AP_segm = result['segm']['AP']
    AP50_segm = result['segm']['AP50']
    AP75_segm = result['segm']['AP75']
    
    return AP_bbox, AP50_bbox, AP75_bbox, AP_segm, AP50_segm, AP75_segm

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        AP_bbox, AP50_bbox, AP75_bbox, AP_segm, AP50_segm, AP75_segm = train_hail_model(hparams, run_dir)
        tf.summary.scalar(AP_BBOX, AP_bbox, step=1)
        tf.summary.scalar(AP50_BBOX, AP50_bbox, step=1)
        tf.summary.scalar(AP75_BBOX, AP75_bbox, step=1)
        tf.summary.scalar(AP_SEGM, AP_segm, step=1)
        tf.summary.scalar(AP50_SEGM, AP50_segm, step=1)
        tf.summary.scalar(AP75_SEGM, AP75_segm, step=1)

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=cfg.OUTPUT_DIR):
    
    return COCOEvaluator(dataset_name, cfg, False, output_folder)

  def build_hooks(self):
        
    hooks = super().build_hooks()
    hooks.insert(-1,LossEvalHook(
        cfg.TEST.EVAL_PERIOD,
        self.model,
        build_detection_test_loader(
            self.cfg,
            self.cfg.DATASETS.TEST[0],
            DatasetMapper(self.cfg,True)
        )
    ))
    return hooks

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
             
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [2]:
#Register train data set
register_coco_instances("train_hail", {}, "./data/hail_20210620_r1/train/annotations/instances_default.json", "./data/hail_20210620_r1/train/images")
dataset_dicts_train = DatasetCatalog.get("train_hail")
hail_metadata_train = MetadataCatalog.get("train_hail")

#Register validation data set
register_coco_instances("val_hail", {}, "./data/hail_20210620_r1/val/annotations/instances_default.json", "./data/hail_20210620_r1/val/images")
dataset_dicts_val = DatasetCatalog.get("val_hail")
hail_metadata_val = MetadataCatalog.get("val_hail")

#Register test data set
register_coco_instances("test_hail", {}, "./data/hail_20210620_r1/test/lai/annotations/instances_default.json", "./data/hail_20210620_r1/test/lai/images")
dataset_dicts_test = DatasetCatalog.get("test_hail")
hail_metadata_test = MetadataCatalog.get("test_hail")

In [4]:
train_cats = []
val_cats = []
for i in gtrain['annotations']:
    j = i['category_id']
    for cat in gtrain['categories']:
        if j == cat['id']:
            train_cats.append(cat['name'])


for k in gval['annotations']:
    w = k['category_id']
    for cat in gval['categories']:
        if w == cat['id']:
            val_cats.append(cat['name'])

# Create dictionary of category and counts
train_count_dict = dict(Counter(train_cats))
val_count_dict = dict(Counter(val_cats))

# Create dictionary of category and ids
ids = []
cats = []
for ind in range(len(gtrain['categories'])):
    cat_id,cat,_ = zip(gtrain['categories'][ind].values())
    ids.append(cat_id[0])
    cats.append(cat[0])
cat_dict = dict(zip(ids, cats))

missing_train = set(list(cat_dict.values())) - set(list(train_count_dict.keys()))
tadd = dict(zip(missing_train, [0]*len(missing_train)))
train_count_dict.update(tadd)

missing_val = set(list(cat_dict.values())) - set(list(val_count_dict.keys()))
vadd = dict(zip(missing_val, [0]*len(missing_val)))
val_count_dict.update(vadd)
    


[{'file_name': './data/hail_20210620_r1/train/images/odm_20210620_r1-00-10.png',
  'height': 500,
  'width': 500,
  'image_id': 1,
  'annotations': [{'iscrowd': 0,
    'bbox': [80.72, 71.55, 6.8, 8.5],
    'category_id': 0,
    'segmentation': [[81.92,
      71.55,
      80.82,
      72.65,
      80.72,
      74.75,
      81.22,
      76.15,
      81.62,
      77.35,
      82.32,
      78.45,
      83.42,
      79.75,
      84.72,
      80.05,
      86.32,
      79.75,
      87.22,
      78.65,
      87.52,
      77.55,
      86.92,
      75.55,
      86.42,
      74.65,
      86.02,
      73.55,
      85.32,
      72.45,
      84.02,
      71.75,
      82.82,
      71.55]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [81.5, 153.0, 6.5, 9.42],
    'category_id': 0,
    'segmentation': [[83.0,
      153.4,
      81.9,
      155.02,
      81.5,
      156.8,
      81.5,
      158.7,
      81.9,
      160.42,
      82.6,
      161.61,
      83.8,
      162.42,
  

In [17]:
count = []
for f in dataset_dicts_train:
    for an in f['annotations']:
        count.append(an['category_id'])
        
print(np.size(count))      

937


In [15]:
np.size(count)

215

In [8]:
#For starting one training run

from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("train_hail",)
cfg.INPUT.MIN_SIZE_TRAIN = (500,)
cfg.INPUT.MIN_SIZE_TRAIN = 0
cfg.OUTPUT_DIR = "output/logs/test/"
cfg.DATASETS.TEST = ("val_hail",)
cfg.DATALOADER.NUM_WORKERS = 1

#cfg.INPUT.RANDOM_FLIP = "horizontal"
#cfg.SOLVER.CHECKPOINT_PERIOD = 2000

cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.0004

cfg.SOLVER.WARMUP_ITERS = 100
cfg.SOLVER.MAX_ITER = 1000 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (700, 800, 900)
cfg.SOLVER.GAMMA = 0.6

# Test
cfg.TEST.EVAL_PERIOD = 200
cfg.TEST.DETECTIONS_PER_IMAGE = 50
#cfg.TEST.EXPECTED_RESULTS = [['bbox', 'AP', 70, 0.5]]

cfg.MODEL.WEIGHTS = "model_final_f10217.pkl"  # initialize from model zoo
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 class (hail)
cfg.MODEL.MASK_ON = True

#os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = CocoTrainer(cfg)
#trainer.resume_or_load(resume=False)
#trainer.train()

In [3]:
#Register train data set
register_coco_instances("train_hail", {}, "./data/hail_20210620_r1/train/annotations/instances_default.json", "./data/hail_20210620_r1/train/images")
dataset_dicts_train = DatasetCatalog.get("train_hail")
hail_metadata_train = MetadataCatalog.get("train_hail")

#Register validation data set
register_coco_instances("val_hail", {}, "./data/hail_20210620_r1/val/annotations/instances_default.json", "./data/hail_20210620_r1/val/images")
dataset_dicts_val = DatasetCatalog.get("val_hail")
hail_metadata_val = MetadataCatalog.get("val_hail")

#Register test data set of lai
register_coco_instances("test_hail_lai", {}, "./data/hail_20210620_r1/test/lai/annotations/instances_default.json", "./data/hail_20210620_r1/test/lai/images")
dataset_dicts_test_lai = DatasetCatalog.get("test_hail_lai")
hail_metadata_test_lai = MetadataCatalog.get("test_hail_lai")

#Register test data set of mon
register_coco_instances("test_hail_mon", {}, "./data/hail_20210620_r1/test/mon/annotations/instances_default.json", "./data/hail_20210620_r1/test/mon/images")
dataset_dicts_test_mon = DatasetCatalog.get("test_hail_mon")
hail_metadata_test_mon = MetadataCatalog.get("test_hail_mon")

#Register test data set of wod
register_coco_instances("test_hail_wod", {}, "./data/hail_20210620_r1/test/wod/annotations/instances_default.json", "./data/hail_20210620_r1/test/wod/images")
dataset_dicts_test_wod = DatasetCatalog.get("test_hail_wod")
hail_metadata_test_wod = MetadataCatalog.get("test_hail_wod")

In [6]:
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

runs = ['run-0','run-1','run-2','run-3','run-4','run-5','run-6','run-7','run-8','run-9','run-10','run-11','run-12','run-13','run-14','run-15']

for run in runs: 
    experiment_folder = './output/logs/hparam_tuning/'+run+'/'

    cfg.MODEL.WEIGHTS = os.path.join(experiment_folder, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
    cfg.TEST.DETECTIONS_PER_IMAGE = 50
    predictor = DefaultPredictor(cfg)

    output_dir = experiment_folder+"evaluation/val/sc005/"

    isExist = os.path.exists(output_dir)
    if not isExist:   
        # Create a new directory because it does not exist 
        os.makedirs(output_dir)
        print("The new directory is created!")

    evaluator = COCOEvaluator("val_hail", cfg, False, output_dir=output_dir)
    val_loader = build_detection_test_loader(cfg, "val_hail")
    inference_on_dataset(predictor.model, val_loader, evaluator)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.948
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.554
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.093
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.622
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.943
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.552
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.616
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


...

The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.913
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.554
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.088
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.593
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.618
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.525
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.914
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.514
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.525
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.092
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.586
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.612
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.525
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.914
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.525
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.525
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.094
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.586
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.614
 Average Recall     (AR) @[ IoU=0.50

In [7]:
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

runs = ['run-0','run-1','run-2','run-3','run-4','run-5','run-6','run-7','run-8','run-9','run-10','run-11','run-12','run-13','run-14','run-15']

for run in runs: 
    experiment_folder = './output/logs/hparam_tuning/'+run+'/'

    cfg.MODEL.WEIGHTS = os.path.join(experiment_folder, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
    cfg.TEST.DETECTIONS_PER_IMAGE = 50
    predictor = DefaultPredictor(cfg)

    output_dir = experiment_folder+"evaluation/test/sc005/"

    isExist = os.path.exists(output_dir)
    if not isExist:   
        # Create a new directory because it does not exist 
        os.makedirs(output_dir)
        print("The new directory is created!")

    evaluator = COCOEvaluator("test_hail_lai", cfg, False, output_dir=output_dir)
    val_loader = build_detection_test_loader(cfg, "test_hail_lai")
    inference_on_dataset(predictor.model, val_loader, evaluator)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.13s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.510
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.898
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.510
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.100
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.562
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.492
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.901
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.493
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.492
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.100
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.560
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.584
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


...

The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.473
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.873
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.473
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.092
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.549
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.574
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.869
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.434
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.096
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.554
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.577
 Average Recall     (AR) @[ IoU=0.50

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.458
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.852
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.450
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.458
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.091
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.538
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.564
 Average Recall     (AR) @[ IoU=0.50

In [6]:
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

experiment_folder = './output/logs/hparam_tuning/run-3/'

cfg.MODEL.WEIGHTS = os.path.join(experiment_folder, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.90
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.TEST.DETECTIONS_PER_IMAGE = 50
predictor = DefaultPredictor(cfg)

output_dir = experiment_folder+"evaluation/test/mon/"

isExist = os.path.exists(output_dir)
if not isExist:   
    # Create a new directory because it does not exist 
    os.makedirs(output_dir)
    print("The new directory is created!")
    
evaluator = COCOEvaluator("test_hail_mon", cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "test_hail_mon")
inference_on_dataset(predictor.model, val_loader, evaluator)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.770
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.242
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.072
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=

OrderedDict([('bbox',
              {'AP': 34.57902815290845,
               'AP50': 77.0260672827717,
               'AP75': 24.22953660330201,
               'APs': 34.57902815290845,
               'APm': nan,
               'APl': nan}),
             ('segm',
              {'AP': 37.61086285329718,
               'AP50': 79.3827498864555,
               'AP75': 31.757579853432027,
               'APs': 37.61086285329718,
               'APm': nan,
               'APl': nan})])

In [7]:
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

experiment_folder = './output/logs/hparam_tuning/run-3/'

cfg.MODEL.WEIGHTS = os.path.join(experiment_folder, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.90
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.TEST.DETECTIONS_PER_IMAGE = 50
predictor = DefaultPredictor(cfg)

output_dir = experiment_folder+"evaluation/test/wod/"

isExist = os.path.exists(output_dir)
if not isExist:   
    # Create a new directory because it does not exist 
    os.makedirs(output_dir)
    print("The new directory is created!")
    
evaluator = COCOEvaluator("test_hail_wod", cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "test_hail_wod")
inference_on_dataset(predictor.model, val_loader, evaluator)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


The new directory is created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.389
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.749
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.354
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.389
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.074
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.429
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.452
 Average Recall     (AR) @[ IoU=0.50

OrderedDict([('bbox',
              {'AP': 38.86432108857008,
               'AP50': 74.90903917496168,
               'AP75': 35.37918223635524,
               'APs': 38.86432108857008,
               'APm': nan,
               'APl': nan}),
             ('segm',
              {'AP': 35.805861644204626,
               'AP50': 73.90257343552909,
               'AP75': 26.483368865401967,
               'APs': 35.805861644204626,
               'APm': nan,
               'APl': nan})])

In [33]:
#Plot test instances
from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
import numpy as np

experiment_folder = './output/logs/hparam_tuning/run-3/'
instance_path = 'products/hparam_tuning/run-3/test/instances/'
dataset = dataset_dicts_test_lai
metadat = hail_metadata_test_lai

if not os.path.exists(instance_path):
    os.makedirs(instance_path)  

#Load configuration
cfg = get_cfg()
cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = os.path.join(experiment_folder, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.90
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.TEST.DETECTIONS_PER_IMAGE = 50

predictor = DefaultPredictor(cfg)

#Plot results on test pictures
for d in dataset:    
    im = cv2.imread(d["file_name"])
    image_name = Path(d["file_name"]).stem
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadat, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    im = v.get_image()
    fig=plt.figure(figsize=[30,30])
    plt.imshow(im)
    plt.savefig(instance_path+'pred_'+image_name+'.png',dpi=300, bbox_inches='tight')
    plt.close()

In [46]:
#Extract all bboxes in test dataset
experiment_folder = './output/logs/hparam_tuning/run-3/'
instance_path = 'products/hparam_tuning/run-3/test/instances/'
dataset = dataset_dicts_test
metadat = hail_metadata_test

bbox_test = []
segm_test = []

for d in dataset:
    for hail in d['annotations']:
        bbox_test.append(hail['bbox'])
        segm_test.append(hail['segmentation'])

In [8]:
#extract masks from test dataset
import pickle
from pycocotools.coco import COCO

user=['mon','wod']

for u in user:
    coco = COCO('./data/hail_20210620_r1/test/'+u+'/annotations/instances_default.json')
    img_dir = './data/hail_20210620_r1/test/'+u+'/images'

    mask_array_path = 'products/annotation/test/hail_20210620_r1_'+u+'/pkl/'

    if not os.path.exists(mask_array_path):
        os.makedirs(mask_array_path)  

    for i in range(1,34):
        img = coco.imgs[i]
        cat_ids = coco.getCatIds()
        anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
        anns = coco.loadAnns(anns_ids)

        mask_array = []
        for i in range(len(anns)):
            mask_array.append(coco.annToMask(anns[i]))

        with open(mask_array_path+'mask_array_'+img['file_name']+'.pkl','wb') as f:
            pickle.dump(mask_array, f)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [9]:
#Detect hail and save segmentation masks
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
import numpy as np

from detectron2.utils.visualizer import ColorMode
import glob
from pathlib import Path
import pickle
import numpy as np

experiment_folder = './output/logs/hparam_tuning/run-3/'

#Load configuration
cfg = get_cfg()
cfg.merge_from_file("/home/appuser/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = os.path.join(experiment_folder, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.90
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.TEST.DETECTIONS_PER_IMAGE = 50

predictor = DefaultPredictor(cfg)

images_path = 'data/hail_20210620_r1/all_images/'
mask_array_path = 'products/hparam_tuning/run-3/all_images/pkl/'

if not os.path.exists(mask_array_path):
    os.makedirs(mask_array_path)  

all_images = glob.glob(images_path+'*.png')
all_images.sort()

for file in all_images:
    im = cv2.imread(file)
    outputs = predictor(im)
    masks = outputs['instances'][outputs['instances'].pred_classes==0].pred_masks.cpu().numpy()

    image_name = Path(file).stem

    mask_array = []
    #hail_count = 1 # For saving all individual masks to a figure
    for i in range(masks.shape[0]):
        mask_int = masks[i,:,:]*1
        mask_array.append(mask_int)

    with open(mask_array_path+'mask_array_'+image_name+'.pkl','wb') as f:
        pickle.dump(mask_array, f)

In [ ]:
#Show random image from train dataset with annotation
for d in random.sample(dataset_dicts_train, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=hail_metadata_train, scale=1)
    vis = visualizer.draw_dataset_dict(d)
    im = vis.get_image()
    fig=plt.figure(figsize=[12,10])
    plt.imshow(im)

In [ ]:
#Download pre-trained model
#!wget https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl
#!wget https://dl.fbaipublicfiles.com/detectron2/new_baselines/mask_rcnn_R_50_FPN_400ep_LSJ/42019571/model_final_14d201.pkl    